## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-29-06-35-24 +0000,nyt,Students Call for More Protests in Indonesia A...,https://www.nytimes.com/2025/08/29/world/asia/...
1,2025-08-29-06-34-04 +0000,nyt,Thai Court to Rule on Paetongtarn Shinawatra’s...,https://www.nytimes.com/2025/08/28/world/asia/...
2,2025-08-29-06-27-58 +0000,bbc,Famous croc wrangler found guilty in evidence ...,https://www.bbc.com/news/articles/c5y0ey8qvw9o...
3,2025-08-29-06-24-37 +0000,bbc,'It's a chaotic mess': UK firms warn over US s...,https://www.bbc.com/news/articles/cvg3yjnq5zeo...
4,2025-08-29-06-24-30 +0000,nyt,"2 Weeks After Trump Talks, Russia Bombards Kyi...",https://www.nytimes.com/2025/08/28/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,46
105,shooting,20
94,minneapolis,14
227,new,13
357,fire,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
123,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,127
227,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,104
210,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,100
236,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...,98
214,2025-08-28-15-14-52 +0000,bbc,Can Trump fire Fed Governor Lisa Cook and why ...,https://www.bbc.com/news/articles/cedvj2d5538o...,98


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
123,127,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
280,60,2025-08-28-09-04-14 +0000,nyt,Survivors of Minneapolis Catholic School Shoot...,https://www.nytimes.com/2025/08/28/us/minneapo...
144,57,2025-08-28-20-32-41 +0000,nypost,"UK, EU accuse Putin of ‘sabotaging peace’ afte...",https://nypost.com/2025/08/28/world-news/uk-eu...
73,45,2025-08-29-00-39-00 +0000,wsj,New Trial Ordered for Three Former Police Offi...,https://www.wsj.com/us-news/law/new-trial-orde...
236,43,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...
45,35,2025-08-29-02-16-31 +0000,nypost,RFK Jr’s deputy expected to replace ousted CDC...,https://nypost.com/2025/08/28/us-news/rfk-jrs-...
72,34,2025-08-29-00-41-41 +0000,nypost,One of 37 intel pros whose clearance was pulle...,https://nypost.com/2025/08/28/us-news/one-of-3...
21,33,2025-08-29-04-14-47 +0000,startribune,Family of boy in Rochester racist attack speak...,https://www.startribune.com/family-of-boy-in-r...
292,32,2025-08-28-08-02-39 +0000,nypost,"Vladimir Putin, Kim Jong Un to attend Chinese ...",https://nypost.com/2025/08/28/world-news/putin...
166,31,2025-08-28-18-13-08 +0000,latimes,ICE will 'ramp up' immigration raids in Los An...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
